In [190]:
#1 Reading in the csv-files
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
movieData= pd.read_csv('./Data/movies.csv', usecols=["movieId", 'title'], dtype={'movieId': 'int16', 'title': 'str'})
ratingData = pd.read_csv('./Data/ratings.csv', usecols=["userId", "movieId", "rating"], dtype ={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})
ratingData.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [191]:
len(ratingData)

27753444

In [ ]:
test = ratingData["userId"].value_counts().sort_values(ascending=True)
for i in range(3):
    ratingData = ratingData.drop(ratingData[ratingData.userId == test.index[i]].index)
len(ratingData)

In [137]:
#Getting rid of some data, trying with only movies rated 5.
ratingData = ratingData.drop(ratingData[ratingData.rating < 5].index)
ratingData = ratingData.drop(ratingData[ratingData.movieId > 500 ].index)
len(ratingData)

661173

In [138]:
ratingData.index

Int64Index([      61,      107,      111,      119,      779,      781,
                 782,      852,      853,      854,
            ...
            27752989, 27752998, 27753001, 27753004, 27753016, 27753023,
            27753028, 27753314, 27753315, 27753334],
           dtype='int64', length=661173)

In [139]:
movies_users=ratingData.pivot(index='movieId', columns='userId', values='rating').fillna(0)
mat_movies_users=csr_matrix(movies_users.values)

In [140]:
model_knn=NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [141]:
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [143]:
def recommender(movie_name, data, model, n_recommendations):
    model.fit(data)
    idx=process.extractOne(movie_name, movieData['title'])[2]
    print("Movie Selected: ", movieData['title'][idx], "Index: ",idx)
    print("Searching for recommendations...")
    distances, indices = model.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    print(movieData.loc[indices[0][1:]])

recommender('Shawshank redemption', mat_movies_users, model_knn, 5)

KeyboardInterrupt: 